## Import Packages

In [2]:
!pip install gensim

     |████████████████████████████████| 24.2 MB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 113 kB 24.4 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=275fda189e5cbc1b9c0b50d75074da1c91d87b30d81af382627a84278b7cc624
  Stored in directory: /home/grant/.cache/pip/wheels/11/73/9a/f91ac1f1816436b16423617c5be5db048697ff152a9c4346f2
Successfully built smart-open


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import spotipy

import json
import gensim
from operator import itemgetter

import sqlite3
import creds

## Define Functions

In [2]:
def find_song(song_name, limit=20):
    c = conn.cursor()
    c.execute("SELECT * FROM songs WHERE UPPER(name) LIKE '%" + song_name + "%'")
    res = sorted(((x, model.wv.vocab[x[0]].count) 
                  for x in c.fetchall() if x[0] in model.wv.vocab), key=itemgetter(-1), reverse = True)
    return [*res][:limit]

In [3]:
def suggest_songs(song_id):
    c = conn.cursor()
    similar = dict(model.wv.most_similar([song_id], topn = 25))
    song_ids = ', '.join(("'%s'" % x) for x in similar.keys())
    c.execute("SELECT * FROM songs WHERE id in (%s)" % song_ids)
    res = sorted((rec + (similar[rec[0]],) for rec in c.fetchall()), 
                 key=itemgetter(-1),
                 reverse=True)
    for i in res:
        print(i)
        
    return list(similar)

## Class to Split Tracks

In [4]:
# Create a class for breaking up model vocab
class WordSplitter(object):
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
        with open(self.filename) as fin:
            for line in fin:
                yield line.split()

## Instantiate Word2Vec Model

In [7]:
# Load Pre-trained model
model = gensim.models.word2vec.Word2Vec.load("data/word2vec.model")

In [5]:
# Instantiate Word2Vec model with songs
model = gensim.models.Word2Vec(WordSplitter("data/songs_ids.txt"), min_count = 100)

In [6]:
# Save Model
model.save("data/word2vec.model")

## Track Identification and Recommendation

In [8]:
# Connect to the database of songs and find frequency for tracks
conn = sqlite3.connect('data/songs.db')

for t in find_song('cold as'):
    print(*t)

('7vidktgNZFQylTgH1GEnMs', 'Cold as Ice', 'Foreigner') 5220
('3iM9wFGlyR28IBZrXYZK9j', 'Cold as Ice - 2008 Remaster', 'Foreigner') 1484
('5jfF26NIqQV3DXMOVgqEli', 'Cold as Stone', 'Kaskade') 884
('493CPxYEHHzXeSdnp8Ul73', 'Cold as Ice', 'M.O.P.') 742
('6chZg1OMXz9VN3MQCr4cNf', 'Cold as Stone - Lipless Remix', 'Kaskade') 553
('6ssiAA2WAtdTgV9WAORq6w', 'As Cold as Ice - F Noizer mix', 'HCMF') 429
('67rvuV6oRGppOyXZk9OUrs', "Cold as Stone - Kaskade's Sunsoaked Mix", 'Kaskade') 361
('7an1exwMnfYRcdVQm0yDev', 'Cold As You', 'Taylor Swift') 358
('5apjl9HKCy9HTHQFivcsOe', 'Cold as Ice', 'Blacklite District') 279
('7rMBg2BQeWk6O7SblbK53z', 'Cold As Ice', 'Starsplash') 237
('7n5d0YesAmy7qlOMhuxzft', 'Cold As Ice (Jumpstyle Edit)', 'Starsplash') 213
('5KirH5QuyG2cybx3fOURIG', 'Cold as Ice', 'Foreigner') 161
('5kPxpQM1fhfi0L0PULZ4wA', 'Cold as Ice', 'M.O.P.') 158
('7IWV29mryrqtO0JVgnPtfv', 'Cold As You', 'Luke Combs') 146
('569sXXQ7t0jSdqHooi2yqs', 'Cold As You', 'Taylor Swift') 126
('5w5R9ShCe6p

In [9]:
# Suggest songs
rec_tracks = suggest_songs('7IWV29mryrqtO0JVgnPtfv')

('3PwtBxiuJGVGHFfHjOjTpY', 'My Kinda Folk', 'Luke Combs', 0.9904252886772156)
('0jcZ1uIHmvh6Is9T6hXOUk', 'The Other Guy', 'Luke Combs', 0.9880845546722412)
('3zxAw8PaqpeGZ3Pakkxv3J', 'Without You', 'Luke Combs', 0.9818929433822632)
('4QhFrBkJ3Rqx66Xs8xwH8o', 'If It Wasn’t For Trucks', 'Riley Green', 0.9640644192695618)
('0nGXi46VcQQ56ZJR428MKS', 'Does To Me (feat. Eric Church)', 'Luke Combs', 0.9440213441848755)
('21HxYsyuuXZNqB1Dme5PQN', 'Wildfire', 'Nate Smith', 0.9431775212287903)
('6ywiHcxaHBR39gRhud37XK', "Young Man's Blues", 'Parker McCollum', 0.9358137249946594)
('3YuvAPidBWldqjagOryo40', 'I Called Mama', 'Tim McGraw', 0.9280024170875549)
('5nra2uQhiaYKKpemgiDNC3', 'GIVE HEAVEN SOME HELL', 'HARDY', 0.9275608658790588)
('0eBXyY4SatzpE7opnzgXvz', 'More Than My Hometown', 'Morgan Wallen', 0.925216019153595)
('5v6gozUHFv9OoNdFYz6p8V', 'Six Feet Apart', 'Luke Combs', 0.9215282201766968)
('34slXA5D75T5S0dwYYRd4J', 'Without You', 'Luke Combs', 0.9203657507896423)
('47qZzTLLJcuD69AIKnqu

## Create Recommended Playlists

In [13]:
# Create Spotify Session
session = creds.session

In [14]:
# Create New Playlist
playlist_details = session.user_playlist_create(user = creds.user_id,
                                                name = "Somewhat Omniscient Neuropath Generator")

# Add Tracks to new playlist
session.user_playlist_add_tracks(user = creds.user_id, 
                                 playlist_id = playlist_details["id"],
                                 tracks = rec_tracks)

{'snapshot_id': 'MywyMmEzMTdiZGUyYjU4YjQzNTJkMzk5ZmM0OWQ2ODJhYWM3NGNhM2Q1'}